In [41]:
import pyspark
from pyspark.sql import SparkSession

In [42]:
csv_path = r"../../dataset/source_folder/csv/books.csv"
multi_csv_path = r"../../dataset/source_folder/csv/"
parquet_path = r"../../dataset/source_folder/parquet/books.parquet"

In [43]:
spark = SparkSession.builder.appName("dataframe extraction").getOrCreate()
spark

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

## Reading CSV file

In [ ]:
df = spark.read\
    .format("csv")\
    .option("schema", "book_id integer, book_title string, book_cost float, book_type string, publ_id integer, book_edition string, book_discount integer, isbn string, publ_date as string")\
    .option("header", True)\
    .option("sep", ",")\
    .option("inferSchema", False)\
    .load(csv_path)
df.show(5, truncate=False)

+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|BOOK_ID|BOOK_TITLE                         |BOOK_COST|BOOK_TYPE|PUBL_ID|BOOK_EDITION|BOOK_DISCOUNT|ISBN          |PUBL_DATE|
+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|B0056  |DataBase Systems                   |935.27   |DB       |11     |Fifth       |10           |978-8131716250|01-May-08|
|B0059  |Database Systems                   |1003.67  |DB       |11     |Eighth      |10           |978-8185015583|01-Jan-02|
|B0063  |The Java TM Programming Language   |425      |PR       |11     |Third       |10           |978-8131702215|01-Jan-08|
|B0067  |The Java TM EE5 Tutorial           |725      |PR       |11     |NULL        |18           |978-8131714928|03-Nov-06|
|B0071  |The Complete Reference HTML & XHTML|540      |PR       |15     |Fourth      |16           |978-0070701946|08-

## Reading Parquet file

In [ ]:
df = spark.read\
    .format("parquet")\
    .option("header", True)\
    .option("inferSchema", True)\
    .load(parquet_path)
df.show(5, truncate=False)

+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|BOOK_ID|BOOK_TITLE                         |BOOK_COST|BOOK_TYPE|PUBL_ID|BOOK_EDITION|BOOK_DISCOUNT|ISBN          |PUBL_DATE|
+-------+-----------------------------------+---------+---------+-------+------------+-------------+--------------+---------+
|B0056  |DataBase Systems                   |935.27   |DB       |11     |Fifth       |10           |978-8131716250|01-May-08|
|B0059  |Database Systems                   |1003.67  |DB       |11     |Eighth      |10           |978-8185015583|01-Jan-02|
|B0063  |The Java TM Programming Language   |425      |PR       |11     |Third       |10           |978-8131702215|01-Jan-08|
|B0067  |The Java TM EE5 Tutorial           |725      |PR       |11     |NULL        |18           |978-8131714928|03-Nov-06|
|B0071  |The Complete Reference HTML & XHTML|540      |PR       |15     |Fourth      |16           |978-0070701946|08-

## Reading JSON file

In [ ]:
data = [{"name": "John", "age": 30, "married": True, "salary": None}]

## Reading Kafka

In [ ]:
df = spark.readStream\
    .format('kafka')\
    .option('kafka.bootstrap.servers', "kafka_bootstrap_servers")\
    .option('subscribe', "kafka_topic")\
    .option('startingoffsets', 'latest')\
    .load()